In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
from sympy.physics.wigner import wigner_3j

# Hand in 3

## NNBC

In [3]:
# Moment of inertia 
I_1 = 10
I_2 = 12
Delta_E = 504.4
I_Gd = 1.1
h = 197.326/(2*np.pi)
M_I = (h**2)*(I_2*(I_2 + 1) - I_1*(I_1 + 1))/(2*Delta_E)
print(M_I)

44.97401352509873


## Clebsch Gordan 

In [9]:
# Coupling between p3/2 and p1/2
j1 = 3/2  # p3/2 state
j2 = 1/2  # p1/2 state

# Total J values possible for this coupling
J_values = [1, 2]  # Result from |j1 - j2| to j1 + j2

# Loop through possible m1, m2, and M (projection of J)
for J in J_values:
    for m1 in [-j1, -j1 + 1, j1 - 1, j1]:
        for m2 in [-j2, j2]:
            M = m1 + m2  # Projection of total angular momentum
            # Calculate Wigner 3-j symbol for the coupling
            wigner_value = wigner_3j(j1, j2, J, m1, m2, -M)
            print(f"J = {J}, m1 = {m1}, m2 = {m2}, M = {M}, Wigner 3-j value = {wigner_value}")

J = 1, m1 = -1.5, m2 = -0.5, M = -2.0, Wigner 3-j value = 0
J = 1, m1 = -1.5, m2 = 0.5, M = -1.0, Wigner 3-j value = -1/2
J = 1, m1 = -0.5, m2 = -0.5, M = -1.0, Wigner 3-j value = sqrt(3)/6
J = 1, m1 = -0.5, m2 = 0.5, M = 0.0, Wigner 3-j value = sqrt(6)/6
J = 1, m1 = 0.5, m2 = -0.5, M = 0.0, Wigner 3-j value = -sqrt(6)/6
J = 1, m1 = 0.5, m2 = 0.5, M = 1.0, Wigner 3-j value = -sqrt(3)/6
J = 1, m1 = 1.5, m2 = -0.5, M = 1.0, Wigner 3-j value = 1/2
J = 1, m1 = 1.5, m2 = 0.5, M = 2.0, Wigner 3-j value = 0
J = 2, m1 = -1.5, m2 = -0.5, M = -2.0, Wigner 3-j value = -sqrt(5)/5
J = 2, m1 = -1.5, m2 = 0.5, M = -1.0, Wigner 3-j value = sqrt(5)/10
J = 2, m1 = -0.5, m2 = -0.5, M = -1.0, Wigner 3-j value = sqrt(15)/10
J = 2, m1 = -0.5, m2 = 0.5, M = 0.0, Wigner 3-j value = -sqrt(10)/10
J = 2, m1 = 0.5, m2 = -0.5, M = 0.0, Wigner 3-j value = -sqrt(10)/10
J = 2, m1 = 0.5, m2 = 0.5, M = 1.0, Wigner 3-j value = sqrt(15)/10
J = 2, m1 = 1.5, m2 = -0.5, M = 1.0, Wigner 3-j value = sqrt(5)/10
J = 2, m1 = 1.5

In [16]:
# Coupling qui m'interesse 
wigner_value = wigner_3j(1/2, 3/2, 2, 1/2, -1/2, 0)
print(f"clebsh 1 = {wigner_value}")
wigner_value = wigner_3j(3/2, 1/2, 2, 1/2, -1/2, 0)
print(f"clebsh 2 = {wigner_value}")

clebsh 1 = -sqrt(10)/10
clebsh 2 = -sqrt(10)/10


## Angular part quadrupole transitions and B(E2)

### Code du Wood Saxon + spin orbit

In [10]:
# Declarartion des constantes
dat = 10**(-3)
n = 1000 #n+1
dr = 0.01
r0 = dr
l = 1
omega = 8.6/h
E0 = 55
m_n = 939.57
V0 = 53.2
R0 = 3.13
a = 0.67
s = 1/2
Lambda = 0.5
V_scen = 1 # 0 if shifted harmonic potential, 1 if Wood Saxon potential

#Fonctions 
def new_r():
    tank = np.zeros(n)
    for i in range(n):
        tank[i] = r0 + dr*i
    return tank
def Enl():
    tank = np.zeros(n)
    for i in range(n):
        tank[i] = h*omega*(2*i + l + 3/2)-E0
    return tank
def V_HO(x):
    return (1/2)*m_n*(omega**2)*(x**2)-E0
def V_Sax(x):
    return -V0/(1 + np.exp((x-R0)/a))
def new_V(r,l,s):
    if (s==1/2):
        c = l/2
    elif(s == -1/2):
        c = -(l+1)/2
    tank = np.zeros([n,n])
    tank2 = np.zeros(n)
    if (V_scen == 0):
        for i in range(n):
            tank[i,i] = V_HO(r[i])
            tank2[i] = tank[i,i]
    if (V_scen == 1):
        for i in range(n):
            tank[i,i] = V_Sax(r[i])
            tank2[i] = tank[i,i]
    for i in range(1,n-1):
        tank[i,i] += (tank2[i+1]-tank2[i-1])*Lambda*c/(r[i]*2*dr)
    return tank

def new_T():
    tank = np.zeros([n,n])
    for i in range(n):
        tank[i,i] = 2/(dr**2)
    for i in range(n-1):
        tank[i,i+1] = -1/(dr**2)
        tank[i+1,i] = -1/(dr**2)
    return tank 

def new_L_square(r,l):
    tank = np.zeros([n,n])
    for i in range(n):
        tank[i,i] = l*(l+1)/(r[i]**2)
    return tank

### Creation des waves functions

In [11]:
# Wavefunctions 
#3/2 wave
r = new_r()
T = new_T()
V = new_V(r,l,s)
L_square = new_L_square(r,l)
H = T + L_square + 2*m_n*V/(h**2)
eigen_value, eigen_state = np.linalg.eig(H)
energies = (h**2)*eigen_value/(2*m_n)
positive_indices = np.where(energies < 0)[0]
energies = energies[positive_indices]
eigen_state = eigen_state[:,positive_indices]
sorted_indices = np.argsort(energies)
energies = energies[sorted_indices]
eigen_state = eigen_state[:,sorted_indices]
wave_3_2 = eigen_state [:,0]
wave_3_2 = wave_3_2/np.sqrt(integrate.simps(wave_3_2**2,r))

#1/2 wave
s = -1/2
V = new_V(r,l,s)
L_square = new_L_square(r,l)
H = T + L_square + 2*m_n*V/(h**2)
eigen_value, eigen_state = np.linalg.eig(H)
energies = (h**2)*eigen_value/(2*m_n)
positive_indices = np.where(energies < 0)[0]
energies = energies[positive_indices]
eigen_state = eigen_state[:,positive_indices]
sorted_indices = np.argsort(energies)
energies = energies[sorted_indices]
eigen_state = eigen_state[:,sorted_indices]
wave_1_2 = eigen_state [:,0]
wave_1_2 = wave_1_2/np.sqrt(integrate.simps(wave_1_2**2,r))

C:\Users\rapha\AppData\Local\Temp\ipykernel_12880\2507595334.py:17: DeprecationWarning: 'scipy.integrate.simps' is deprecated in favour of 'scipy.integrate.simpson' and will be removed in SciPy 1.14.0
  wave_3_2 = wave_3_2/np.sqrt(integrate.simps(wave_3_2**2,r))
C:\Users\rapha\AppData\Local\Temp\ipykernel_12880\2507595334.py:33: DeprecationWarning: 'scipy.integrate.simps' is deprecated in favour of 'scipy.integrate.simpson' and will be removed in SciPy 1.14.0
  wave_1_2 = wave_1_2/np.sqrt(integrate.simps(wave_1_2**2,r))


### Calculation quadrupole transition and BE2

In [15]:
# Declaration des constantes 
j_i = 3/2
j_f = 1/2
l_i = 1
l_f = 1
Lambda_quadra= 2
clebsh1 = -np.sqrt(10)/10 # Faire fonctionner code en haut
clebsh2 = -np.sqrt(10)/10

# Declaration des fonctions 
def quadruple_transition(j_i_,j_f_,l_f_,l_i_,Lambda_quadra_,clebsh_,R_lambda_):
        return (np.e/np.sqrt(4*np.pi))*(-1)**(j_i_ + Lambda_quadra_ -1/2)*((1 + (-1)**(l_f_ + l_i_ + Lambda_quadra_))/2)*np.sqrt(2*Lambda_quadra_ + 1)*np.sqrt(2*j_f_ + 1)*np.sqrt(2*j_i_ + 1)*clebsh_*R_lambda_

def B_E2 (j_i_,j_f_,l_f_,l_i_,Lambda_quadra_,clebsh_,R_lambda_):
    return (quadruple_transition(j_f_,j_i_,l_i_,l_f_,Lambda_quadra_,clebsh_,R_lambda_)**2)/(2*j_i_ + 1)

# Calculation of R_lambda
product = wave_1_2*wave_1_2*r**(2 + Lambda_quadra)
R_lambda = integrate.simps(product,r)

# Calculation of quadrople transition and BE2 energy
f_Q_i = quadruple_transition(j_i,j_f,l_f,l_i,Lambda_quadra,clebsh1,R_lambda)
B_EL_Ji_Jf = B_E2(j_i,j_f,l_f,l_i,Lambda_quadra,clebsh2,R_lambda)

print(f"f_Q_i = {f_Q_i}, B_EL_Ji_Jf = {B_EL_Ji_Jf}")

f_Q_i = 4.423356404951047, B_EL_Ji_Jf = 4.891520471305363


C:\Users\rapha\AppData\Local\Temp\ipykernel_12880\3538634003.py:19: DeprecationWarning: 'scipy.integrate.simps' is deprecated in favour of 'scipy.integrate.simpson' and will be removed in SciPy 1.14.0
  R_lambda = integrate.simps(product,r)
